In [ ]:
!pip install pytorch-widedeep

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [13]:
# Korean Movie Recommender system Dataset
# https://github.com/lovit/kmrd

data_path = "C:/Users/kt NexR/PycharmProjects/Recommendation-algorithms/Deep Learning_recommendation/data/kmrd-master"
print (os.getcwd()) #현재 경로 출력

C:\Users\Jamie\j_notebook


In [14]:
if not os.path.exists(data_path):
  !git clone https://github.com/lovit/kmrd
  !python setup.py install
else:
  print("data and path already exists!")

path = data_path + '/kmr_dataset/datafile/kmrd-small'

data and path already exists!


In [15]:
df = pd.read_csv(path+'/rates.csv')
train_df, val_df = train_test_split(df, test_size=0.2, random_state=1234, shuffle=True)

In [17]:
train_df.shape

(112568, 4)

In [18]:
train_df = train_df[:1000]

In [19]:
# Load all related dataframe
movies_df = pd.read_csv(os.path.join(path, 'movies.txt'), sep='\t', encoding='utf-8')
movies_df = movies_df.set_index('movie')

castings_df = pd.read_csv(os.path.join(path, 'castings.csv'), encoding='utf-8')
countries_df = pd.read_csv(os.path.join(path, 'countries.csv'), encoding='utf-8')
genres_df = pd.read_csv(os.path.join(path, 'genres.csv'), encoding='utf-8')

# Get genre information
genres = [(list(set(x['movie'].values))[0], '/'.join(x['genre'].values)) for index, x in genres_df.groupby('movie')]
combined_genres_df = pd.DataFrame(data=genres, columns=['movie', 'genres'])
combined_genres_df = combined_genres_df.set_index('movie')

# Get castings information
castings = [(list(set(x['movie'].values))[0], x['people'].values) for index, x in castings_df.groupby('movie')]
combined_castings_df = pd.DataFrame(data=castings, columns=['movie','people'])
combined_castings_df = combined_castings_df.set_index('movie')

# Get countries for movie information
countries = [(list(set(x['movie'].values))[0], ','.join(x['country'].values)) for index, x in countries_df.groupby('movie')]
combined_countries_df = pd.DataFrame(data=countries, columns=['movie', 'country'])
combined_countries_df = combined_countries_df.set_index('movie')

movies_df = pd.concat([movies_df, combined_genres_df, combined_castings_df, combined_countries_df], axis=1)

print(movies_df.shape)
print(movies_df.head())

(999, 7)
                      title                           title_eng    year  \
movie                                                                     
10001                시네마 천국              Cinema Paradiso , 1988  2013.0   
10002              빽 투 더 퓨쳐           Back To The Future , 1985  2015.0   
10003            빽 투 더 퓨쳐 2    Back To The Future Part 2 , 1989  2015.0   
10004            빽 투 더 퓨쳐 3  Back To The Future Part III , 1990  1990.0   
10005  스타워즈 에피소드 4 - 새로운 희망                    Star Wars , 1977  1997.0   

         grade         genres  \
movie                           
10001   전체 관람가     드라마/멜로/로맨스   
10002  12세 관람가         SF/코미디   
10003  12세 관람가         SF/코미디   
10004   전체 관람가  서부/SF/판타지/코미디   
10005       PG   판타지/모험/SF/액션   

                                                  people   country  
movie                                                               
10001  [4374, 178, 3241, 47952, 47953, 19538, 18991, ...  이탈리아,프랑스  
10002    [1076, 4603, 917,

In [20]:
movies_df.columns

Index(['title', 'title_eng', 'year', 'grade', 'genres', 'people', 'country'], dtype='object')

In [39]:
dummy_genres_df = movies_df['genres'].str.get_dummies(sep='/')
train_genres_df = train_df['movie'].apply(lambda x: dummy_genres_df.loc[x])
train_genres_df.head()

C:\ProgramData\Anaconda3\envs\rc\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,SF,가족,공포,느와르,다큐멘터리,드라마,로맨스,멜로,모험,뮤지컬,...,범죄,서부,서사,스릴러,애니메이션,액션,에로,전쟁,코미디,판타지
137023,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
92868,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
94390,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22289,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
80155,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [40]:
dummy_grade_df = pd.get_dummies(movies_df['grade'], prefix='grade')
train_grade_df = train_df['movie'].apply(lambda x: dummy_grade_df.loc[x])
train_grade_df.head()

C:\ProgramData\Anaconda3\envs\rc\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,grade_12세 관람가,grade_15세 관람가,grade_G,grade_NR,grade_PG,grade_PG-13,grade_R,grade_전체 관람가,grade_청소년 관람불가
137023,1,0,0,0,0,0,0,0,0
92868,0,0,0,0,1,0,0,0,0
94390,1,0,0,0,0,0,0,0,0
22289,0,0,0,0,0,0,0,1,0
80155,1,0,0,0,0,0,0,0,0


In [41]:
train_df['year'] = train_df.apply(lambda x: movies_df.loc[x['movie']]['year'], axis=1)

C:\ProgramData\Anaconda3\envs\rc\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user,movie,rate,time,year,grade_12세 관람가,grade_15세 관람가,grade_G,grade_NR,grade_PG,...,범죄,서부,서사,스릴러,애니메이션,액션,에로,전쟁,코미디,판타지
137023,48423,10764,10,1212241560,1987.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92868,17307,10170,10,1122185220,1985.0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
94390,18180,10048,10,1573403460,2016.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22289,1498,10001,9,1432684500,2013.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80155,12541,10022,10,1370458140,1980.0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2870,86,10334,10,1258175340,1998.0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
120892,34595,10200,10,1571998620,2019.0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
93371,17597,10048,10,1359533220,2016.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58284,6588,10249,10,1090721280,2015.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [42]:
train_df = pd.concat([train_df, train_grade_df, train_genres_df], axis=1)
train_df.head()

C:\ProgramData\Anaconda3\envs\rc\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user,movie,rate,time,year,grade_12세 관람가,grade_15세 관람가,grade_G,grade_NR,grade_PG,...,범죄,서부,서사,스릴러,애니메이션,액션,에로,전쟁,코미디,판타지
137023,48423,10764,10,1212241560,1987.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92868,17307,10170,10,1122185220,1985.0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
94390,18180,10048,10,1573403460,2016.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22289,1498,10001,9,1432684500,2013.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80155,12541,10022,10,1370458140,1980.0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [43]:
wide_cols = list(dummy_genres_df.columns) + list(dummy_grade_df.columns)
wide_cols

C:\ProgramData\Anaconda3\envs\rc\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['SF',
 '가족',
 '공포',
 '느와르',
 '다큐멘터리',
 '드라마',
 '로맨스',
 '멜로',
 '모험',
 '뮤지컬',
 '미스터리',
 '범죄',
 '서부',
 '서사',
 '스릴러',
 '애니메이션',
 '액션',
 '에로',
 '전쟁',
 '코미디',
 '판타지',
 'grade_12세 관람가',
 'grade_15세 관람가',
 'grade_G',
 'grade_NR',
 'grade_PG',
 'grade_PG-13',
 'grade_R',
 'grade_전체 관람가',
 'grade_청소년 관람불가']

In [28]:
print(len(wide_cols))
print(wide_cols)

wide_cols = wide_cols[:3]

30
['SF', '가족', '공포', '느와르', '다큐멘터리', '드라마', '로맨스', '멜로', '모험', '뮤지컬', '미스터리', '범죄', '서부', '서사', '스릴러', '애니메이션', '액션', '에로', '전쟁', '코미디', '판타지', 'grade_12세 관람가', 'grade_15세 관람가', 'grade_G', 'grade_NR', 'grade_PG', 'grade_PG-13', 'grade_R', 'grade_전체 관람가', 'grade_청소년 관람불가']


In [29]:
# wide_cols = ['genre', 'grade']
# cross_cols = [('genre', 'grade')]
wide_cols

['SF', '가족', '공포']

In [31]:
import itertools
from itertools import product
unique_combinations = list(list(zip(wide_cols, element))
                           for element in product(wide_cols, repeat = len(wide_cols)))

print(unique_combinations)
cross_cols = [item for sublist in unique_combinations for item in sublist]
cross_cols = [x for x in cross_cols if x[0] != x[1]]
cross_cols = list(set(cross_cols))
print(cross_cols)

[[('SF', 'SF'), ('가족', 'SF'), ('공포', 'SF')], [('SF', 'SF'), ('가족', 'SF'), ('공포', '가족')], [('SF', 'SF'), ('가족', 'SF'), ('공포', '공포')], [('SF', 'SF'), ('가족', '가족'), ('공포', 'SF')], [('SF', 'SF'), ('가족', '가족'), ('공포', '가족')], [('SF', 'SF'), ('가족', '가족'), ('공포', '공포')], [('SF', 'SF'), ('가족', '공포'), ('공포', 'SF')], [('SF', 'SF'), ('가족', '공포'), ('공포', '가족')], [('SF', 'SF'), ('가족', '공포'), ('공포', '공포')], [('SF', '가족'), ('가족', 'SF'), ('공포', 'SF')], [('SF', '가족'), ('가족', 'SF'), ('공포', '가족')], [('SF', '가족'), ('가족', 'SF'), ('공포', '공포')], [('SF', '가족'), ('가족', '가족'), ('공포', 'SF')], [('SF', '가족'), ('가족', '가족'), ('공포', '가족')], [('SF', '가족'), ('가족', '가족'), ('공포', '공포')], [('SF', '가족'), ('가족', '공포'), ('공포', 'SF')], [('SF', '가족'), ('가족', '공포'), ('공포', '가족')], [('SF', '가족'), ('가족', '공포'), ('공포', '공포')], [('SF', '공포'), ('가족', 'SF'), ('공포', 'SF')], [('SF', '공포'), ('가족', 'SF'), ('공포', '가족')], [('SF', '공포'), ('가족', 'SF'), ('공포', '공포')], [('SF', '공포'), ('가족', '가족'), ('공포', 'SF')], [('SF', '공포'), ('가족', '가족'), ('

In [32]:
# embed_cols = [('genre', 16),('grade', 16)]
embed_cols = list(set([(x[0], 16) for x in cross_cols]))
continuous_cols = ['year']

print(embed_cols)
print(continuous_cols)

[('가족', 16), ('공포', 16), ('SF', 16)]
['year']


In [33]:
target = train_df['rate'].apply(lambda x: 1 if x > 9 else 0).values

In [36]:
from pytorch_widedeep.preprocessing import WidePreprocessor, DensePreprocessor
from pytorch_widedeep.models import Wide, DeepDense, WideDeep
from pytorch_widedeep.metrics import Accuracy

C:\ProgramData\Anaconda3\envs\rc\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


ImportError: cannot import name 'DensePreprocessor' from 'pytorch_widedeep.preprocessing' (C:\ProgramData\Anaconda3\envs\rc\lib\site-packages\pytorch_widedeep\preprocessing\__init__.py)

In [ ]:
preprocess_wide = WidePreprocessor(wide_cols=wide_cols, crossed_cols=cross_cols)
X_wide = preprocess_wide.fit_transform(train_df)
wide = Wide(wide_dim=np.unique(X_wide).shape[0], pred_dim=1)

In [ ]:
X_wide.size

In [ ]:
preprocess_deep = DensePreprocessor(embed_cols=embed_cols, continuous_cols=continuous_cols)
X_deep = preprocess_deep.fit_transform(train_df)
deepdense = DeepDense(
    hidden_layers=[64, 32],
    deep_column_idx=preprocess_deep.deep_column_idx,
    embed_input=preprocess_deep.embeddings_input,
    continuous_cols=continuous_cols,
)

In [ ]:
deepdense

In [ ]:
# build, compile and fit
model = WideDeep(wide=wide, deepdense=deepdense)
model.compile(method="binary", metrics=[Accuracy])
model.fit(
    X_wide=X_wide,
    X_deep=X_deep,
    target=target,
    n_epochs=5,
    batch_size=256,
    val_split=0.1,
)

In [ ]:
X_deep.shape

In [ ]:
X_wide.shape